<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scrape-Illinois---County-Dropoff-Locations" data-toc-modified-id="Scrape-Illinois---County-Dropoff-Locations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scrape Illinois - County Dropoff Locations</a></span></li><li><span><a href="#Analyze-the-format-of-the-text" data-toc-modified-id="Analyze-the-format-of-the-text-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analyze the format of the text</a></span></li><li><span><a href="#Download-html-code-locally-to-scrape" data-toc-modified-id="Download-html-code-locally-to-scrape-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download html code locally to scrape</a></span></li><li><span><a href="#Scrape-address-info" data-toc-modified-id="Scrape-address-info-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scrape address info</a></span><ul class="toc-item"><li><span><a href="#Case-1:-no-address_2" data-toc-modified-id="Case-1:-no-address_2-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Case 1: no address_2</a></span></li><li><span><a href="#Case-2:-address_2-present" data-toc-modified-id="Case-2:-address_2-present-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Case 2: address_2 present</a></span></li><li><span><a href="#Loop-through-class=infoContainers" data-toc-modified-id="Loop-through-class=infoContainers-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Loop through class=infoContainers</a></span></li></ul></li><li><span><a href="#Scrape-schedules" data-toc-modified-id="Scrape-schedules-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Scrape schedules</a></span></li><li><span><a href="#Join-address-and-schedules" data-toc-modified-id="Join-address-and-schedules-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Join address and schedules</a></span></li><li><span><a href="#Create-a-DataFrame-to-hold-the-data" data-toc-modified-id="Create-a-DataFrame-to-hold-the-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Create a DataFrame to hold the data</a></span></li><li><span><a href="#Write-dropoff-data-to-csv-file" data-toc-modified-id="Write-dropoff-data-to-csv-file-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Write dropoff data to csv file</a></span></li><li><span><a href="#Read-csv-of-Illinois-jurisdicitons" data-toc-modified-id="Read-csv-of-Illinois-jurisdicitons-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Read csv of Illinois jurisdicitons</a></span></li><li><span><a href="#Write-jurisdiciton-data-to-csv-file" data-toc-modified-id="Write-jurisdiciton-data-to-csv-file-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Write jurisdiciton data to csv file</a></span></li></ul></div>

# Scrape Illinois - County Dropoff Locations

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = 'https://www.elections.il.gov/VotingAndRegistrationSystems/VoteByMailBallotDropBoxLocations.aspx?MID=0DjRa0Y9V6U%3d&T=637368952002106087'

In [3]:
res = requests.get(url)

In [4]:
res.status_code

200

In [5]:
soup = BeautifulSoup(res.content)

# Analyze the format of the text

In [6]:
counties = soup.find_all(class_='infoContainers')

In [7]:
len(counties)

1

In [8]:
counties

[<div class="infoContainers width-66">
 <div class="infoHeaders">
             Select a Jurisdiction
         </div>
 <div class="infoContent text-center margin-Auto">
 <div class="search-options-container">
 <div class="search-errors">
 </div>
 <div class="left">
                     Election:
                 </div>
 <div class="right">
 <select id="ContentPlaceHolder1_ddlElection" name="ctl00$ContentPlaceHolder1$ddlElection">
 <option selected="selected" value="0">Please Select an Election</option>
 <option value="66">2020 General Election</option>
 </select>
 </div>
 <div class="left">
                     Jurisdiction:
                 </div>
 <div class="right">
 <select id="ContentPlaceHolder1_ddlCounty" name="ctl00$ContentPlaceHolder1$ddlCounty">
 <option selected="selected" value="0">Please Choose a Jurisdiction</option>
 <option value="0">All Jurisdictions</option>
 <option value="1">Adams County</option>
 <option value="2">Alexander County</option>
 <option value="3">Bond Co

It looks like there is some code on this page that calls the drop off locations once the user has selected a county, or all counties, so I am selecting all counties and copying the html container that has the detailed addresses and rebuilding it locally so I can scrape them.

# Download html code locally to scrape

In [9]:
from requests_file import FileAdapter

In [10]:
s = requests.Session()
s.mount('file://', FileAdapter())

res = s.get('file:///Users/seymourliao/Documents/Data Science/hackForLA/ballot_nav/web_scrape_IL/index.html')

In [11]:
res

<Response [200]>

In [12]:
soup = BeautifulSoup(res.content)

# Scrape address info

In [13]:
divs = soup.find_all(class_='infoContainers')

In [14]:
len(divs)

176

In [15]:
divs[1:5]

[<div class="infoContainers width-66"><div class="infoHeaders text-Center"><asp:label runat="server">Adams County</asp:label><br/><asp:label runat="server">County Clerk's Office</asp:label><br/><asp:label runat="server">507 Vermont ST<br/>Quincy, IL  62301</asp:label></div></div>,
 <div class="infoContainers width-66"><div class="infoHeaders text-Center"><asp:label runat="server">Brown County</asp:label><br/><asp:label runat="server">Brown County Clerk’s Office</asp:label><br/><asp:label runat="server">200 Court Street<br/>Room 4<br/>Mt. Sterling, IL  62353</asp:label></div></div>,
 <div class="infoContainers width-66"><div class="infoHeaders text-Center"><asp:label runat="server">Carroll County</asp:label><br/><asp:label runat="server">Carroll County Courthouse</asp:label><br/><asp:label runat="server">301 N Main Street<br/>Mount Carroll, IL  61053</asp:label></div></div>,
 <div class="infoContainers width-66"><div class="infoHeaders text-Center"><asp:label runat="server">Cass County<

In [16]:
type(divs)

bs4.element.ResultSet

In [17]:
divs[1]

<div class="infoContainers width-66"><div class="infoHeaders text-Center"><asp:label runat="server">Adams County</asp:label><br/><asp:label runat="server">County Clerk's Office</asp:label><br/><asp:label runat="server">507 Vermont ST<br/>Quincy, IL  62301</asp:label></div></div>

In [18]:
type(divs[1])

bs4.element.Tag

In [19]:
re.findall('server">(.*?)</', str(divs[1]))

['Adams County',
 "County Clerk's Office",
 '507 Vermont ST<br/>Quincy, IL  62301']

## Case 1: no address_2

In [20]:
re.findall('server">(.*?)</', str(divs[1]))[-1].split('<br/>')

['507 Vermont ST', 'Quincy, IL  62301']

In [21]:
re.findall('server">(.*?)</', str(divs[1]))[-1].split('<br/>')[0]

'507 Vermont ST'

In [22]:
re.findall('server">(.*?)</', str(divs[1]))[-1].split('<br/>')[-1].split(',')[0]

'Quincy'

In [23]:
re.findall('server">(.*?)</', str(divs[1]))[-1].split('<br/>')[-1].split('IL ')[1].strip()

'62301'

## Case 2: address_2 present

In [24]:
re.findall('server">(.*?)</', str(divs[2]))

['Brown County',
 'Brown County Clerk’s Office',
 '200 Court Street<br/>Room 4<br/>Mt. Sterling, IL  62353']

In [25]:
re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')

['200 Court Street', 'Room 4', 'Mt. Sterling, IL  62353']

In [26]:
re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')[0]

'200 Court Street'

In [27]:
re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')[-1]

'Mt. Sterling, IL  62353'

In [28]:
re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')[-1].split('IL ')[1].strip()

'62353'

In [29]:
bool(re.search('IL', re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')[2]))

True

In [30]:
re.findall('server">(.*?)</', str(divs[2]))[-1].split('<br/>')[1].strip()

'Room 4'

## Loop through class=infoContainers

In [31]:
all_addresses = []
for div in divs[1:]:
    county = re.findall('server">(.*?)</', str(div))[0]
    location_type = re.findall('server">(.*?)</', str(div))[1]
    address_1 = re.findall('server">(.*?)</', str(div))[-1].split('<br/>')[0]
    city = re.findall('server">(.*?)</', str(div))[-1].split('<br/>')[-1].split(',')[0]
    zip_code = re.findall('server">(.*?)</', str(div))[-1].split('<br/>')[-1].split('IL ')[1].strip()
    
    if re.search('IL', re.findall('server">(.*?)</', str(div))[-1].split('<br/>')[1]):
        address_2 = np.nan
    else:
        address_2 = re.findall('server">(.*?)</', str(div))[-1].split('<br/>')[1].strip()
    
    data = {'county': county, 'location_type': location_type, 'address_1': address_1, 
            'address_2': address_2, 'city': city, 'zip': zip_code}
    all_addresses.append(data)

In [32]:
all_addresses[-3:]

[{'county': 'Woodford County',
  'location_type': "County Clerk's Office",
  'address_1': '115 N Main St',
  'address_2': 'Rm 104',
  'city': 'Eureka',
  'zip': '61530'},
 {'county': 'City of Galesburg',
  'location_type': 'City Election Office (City Hall)',
  'address_1': '55 w Tompkins St',
  'address_2': nan,
  'city': 'Galesburg',
  'zip': '61401'},
 {'county': 'City of Rockford',
  'location_type': 'Rockford Election Commssion Office',
  'address_1': '301 S 6th ST',
  'address_2': nan,
  'city': 'Rockford',
  'zip': '61104'}]

In [33]:
len(all_addresses)

175

In [34]:
all_addresses_df = pd.DataFrame(all_addresses)

In [35]:
all_addresses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   county         175 non-null    object
 1   location_type  175 non-null    object
 2   address_1      175 non-null    object
 3   address_2      47 non-null     object
 4   city           175 non-null    object
 5   zip            175 non-null    object
dtypes: object(6)
memory usage: 8.3+ KB


# Scrape schedules

In [36]:
schedules = soup.find_all(class_='gridview-container')

In [37]:
len(schedules)

175

In [38]:
all_addresses_df[all_addresses_df['county']=='Cook County']

,county,location_type,address_1,address_2,city,zip
15,Cook County,Cook County Clerk's Main Office,69 W Washington St,Pedway,Chicago,60602
16,Cook County,Palos Heights Recreation Center,6601 W 127th St,NaN,Palos Heights,60463
17,Cook County,Arlington Heights Village Hall,33 S Arlington Heights Rd,NaN,Arlington Heights,60005
18,Cook County,Barrington Hills Village Hall,112 Algonquin Rd,NaN,Barrington Hills,60010
19,Cook County,Bellwood Village Hall,3200 Washington Blvd,NaN,Bellwood,60104
20,Cook County,Burbank (Prairie Trails Library District),8449 S Moody Ave,NaN,Burbank,60459
21,Cook County,Bridgeview Courthouse,10220 S 76th Ave,Room 238,Bridgeview,60455
22,Cook County,Calumet City Public Library,660 S Manistee Ave,NaN,Calumet City,60827
23,Cook County,Calumet Township Community Center,12633 S Ashland Ave,NaN,Calumet Park,60827
24,Cook County,Chicago Heights (Prairie State College),202 S Halsted St,ROOM 1318,Chicago Heights,60411


In [39]:
schedules[0]

<div class="gridview-container"><div>
<table border="1" cellpadding="0" cellspacing="0" class="SearchListTable" id="ContentPlaceHolder1_gvJurisdiction_0" rules="all" style="background-color:White;width:66%;border-collapse:collapse;">
<tbody><tr class="SearchListTableHeaderRow">
<th class="GridViewSortHeaderNone" scope="col">Location Type</th><th class="GridViewSortHeaderNone" scope="col">Dates Of Operation</th><th class="GridViewSortHeaderNone" scope="col">Hours Of Operation</th><th class="GridViewSortHeaderNone" scope="col">Notes</th>
</tr><tr>
<td align="left" style="width:25%;" valign="middle">VBM Ballot Drop Box</td><td align="left" style="width:25%;" valign="middle">September 24, 2020 and concludes November 3, 2020</td><td align="left" style="width:25%;" valign="middle">24 hours/7 Days a week</td><td align="left" style="width:25%;" valign="middle">On November 3rd, drop box closed at 7:00 PM</td>
</tr>
</tbody></table>
</div></div>

In [40]:
schedules[15].find_all('td')

[<td align="left" style="width:25%;" valign="middle">VBM Ballot Drop Box</td>,
 <td align="left" style="width:25%;" valign="middle">November 3, 2020</td>,
 <td align="left" style="width:25%;" valign="middle">Tuesday 6:00 AM to 7:00 PM</td>,
 <td align="left" style="width:25%;" valign="middle">Open to All Voters in Suburban Cook County</td>,
 <td align="left" style="width:25%;" valign="middle">VBM Ballot Drop Box</td>,
 <td align="left" style="width:25%;" valign="middle">Oct 7 through November 2, 2020</td>,
 <td align="left" style="width:25%;" valign="middle">Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm</td>,
 <td align="left" style="width:25%;" valign="middle"> </td>,
 <td align="left" style="width:25%;" valign="middle">VBM Ballot Drop Box</td>,
 <td align="left" style="width:25%;" valign="middle">October 7th through Nov 2, 2020</td>,
 <td align="left" style="width:25%;" valign="middle">Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm</td>,
 <td align=

In [41]:
schedule_list = [re.findall('>(.*?)</td>', str(x.find_all('td'))) for x in schedules]

In [42]:
len(schedule_list)

175

In [43]:
schedule_list[15]

['VBM Ballot Drop Box',
 'November 3, 2020',
 'Tuesday 6:00 AM to 7:00 PM',
 'Open to All Voters in Suburban Cook County',
 'VBM Ballot Drop Box',
 'Oct 7 through November 2, 2020',
 'Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
 '\xa0',
 'VBM Ballot Drop Box',
 'October 7th through Nov 2, 2020',
 'Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
 'Pedway Room']

In [44]:
len(schedule_list[-4])

76

In [45]:
range(int(len(schedules[-4].find_all('td'))/4))

range(0, 19)

In [46]:
all_addresses[0]['county']

'Adams County'

In [47]:
all_schedueles = []
for count,records in enumerate(schedule_list):
    for record in range(int(len(records)/4)):
        county = all_addresses[count]['county']
        address_1 = all_addresses[count]['address_1']
        hours = records[1+record*4] + ': ' + records[2+record*4]
        notes = records[3+record*4]
        data = {'county':county, 'address_1':address_1, 'hours':hours, 'notes':notes}
        all_schedueles.append(data)

In [48]:
len(all_schedueles)

333

In [49]:
schedule_list[15]

['VBM Ballot Drop Box',
 'November 3, 2020',
 'Tuesday 6:00 AM to 7:00 PM',
 'Open to All Voters in Suburban Cook County',
 'VBM Ballot Drop Box',
 'Oct 7 through November 2, 2020',
 'Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
 '\xa0',
 'VBM Ballot Drop Box',
 'October 7th through Nov 2, 2020',
 'Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
 'Pedway Room']

In [50]:
all_schedueles[15:18]

[{'county': 'Cook County',
  'address_1': '69 W Washington St',
  'hours': 'November 3, 2020: Tuesday 6:00 AM to 7:00 PM',
  'notes': 'Open to All Voters in Suburban Cook County'},
 {'county': 'Cook County',
  'address_1': '69 W Washington St',
  'hours': 'Oct 7 through November 2, 2020: Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
  'notes': '\xa0'},
 {'county': 'Cook County',
  'address_1': '69 W Washington St',
  'hours': 'October 7th through Nov 2, 2020: Monday-Friday 8:30am-7:00pm Saturdays and Sundays 9:00am-5:00pm',
  'notes': 'Pedway Room'}]

In [51]:
all_schedueles_df = pd.DataFrame(all_schedueles)

In [52]:
all_schedueles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     333 non-null    object
 1   address_1  333 non-null    object
 2   hours      333 non-null    object
 3   notes      333 non-null    object
dtypes: object(4)
memory usage: 10.5+ KB


# Join address and schedules

In [53]:
dropOff_df = pd.merge(all_addresses_df, all_schedueles_df, on=['county','address_1'])

In [54]:
dropOff_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   county         336 non-null    object
 1   location_type  336 non-null    object
 2   address_1      336 non-null    object
 3   address_2      70 non-null     object
 4   city           336 non-null    object
 5   zip            336 non-null    object
 6   hours          336 non-null    object
 7   notes          336 non-null    object
dtypes: object(8)
memory usage: 23.6+ KB


In [55]:
dropOff_df.head()

,county,location_type,address_1,address_2,city,zip,hours,notes
0,Adams County,County Clerk's Office,507 Vermont ST,NaN,Quincy,62301,"September 24, 2020 and concludes November 3, 2...","On November 3rd, drop box closed at 7:00 PM"
1,Brown County,Brown County Clerk’s Office,200 Court Street,Room 4,Mt. Sterling,62353,"September 24 - November 3, 2020 7:00 PM: 24 Hours",Right side of Courthouse Entrance
2,Carroll County,Carroll County Courthouse,301 N Main Street,NaN,Mount Carroll,61053,"September 24, 2020 thru November 3, 2020: 24 h...",Drop Box will be attached to building by the f...
3,Cass County,Cass County Clerk's Office,100 E Springfield St,NaN,Virginia,62691,"Begins on September 24, 2020 and concludes on ...",Attached to the building just to the right of ...
4,Champaign County,Champaign County Clerk's Office,1776 E Washington St,NaN,Urbana,61802,"Opens September 26, 2020: Open 24 Hours a Day,...",Ballots must be deposited by 7PM on Election D...


In [56]:
dropOff_df.tail()

,county,location_type,address_1,address_2,city,zip,hours,notes
331,Will County,White Oak Library,121 E Eighth ST,NaN,Lockport,60441,"November 1, 2020: Sunday 1:00 PM to 5:00 PM",Right side of circulation desk inside main ent...
332,Will County,White Oak Library,121 E Eighth ST,NaN,Lockport,60441,"November 2, 2020: Monday 10:00 AM to 8:30 PM",Right side of circulation desk inside main ent...
333,Woodford County,County Clerk's Office,115 N Main St,Rm 104,Eureka,61530,"Begins on September 24, 2020 and concluding on...",
334,City of Galesburg,City Election Office (City Hall),55 w Tompkins St,NaN,Galesburg,61401,"September 24th through November 3, 2020: 24/7",A secure ballot drop box is located on the Cit...
335,City of Rockford,Rockford Election Commssion Office,301 S 6th ST,NaN,Rockford,61104,October 1 thru November 3: 24 hrs. Ends Novemb...,City of Rockford Residents Only


# Create a DataFrame to hold the data

In [57]:
data_schema = [
        'address_1',
        'state',
        'county',
        'Field 23',
        'location_type',
        'address_2',
        'city',
        'state_2',
        'zip',
        'phone', 
        'latitude', 
        'longitude', 
        'has_droppff', 
        'has_phone_number', 
        'county_website_url', 
        'validate_url', 
        'email', 
        'fax', 
        'social', 
        'inactive', 
        'hours',
        'rules',
        'notes'
]

In [58]:
all_data_df = pd.DataFrame(columns=data_schema)

In [59]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   address_1           0 non-null      object
 1   state               0 non-null      object
 2   county              0 non-null      object
 3   Field 23            0 non-null      object
 4   location_type       0 non-null      object
 5   address_2           0 non-null      object
 6   city                0 non-null      object
 7   state_2             0 non-null      object
 8   zip                 0 non-null      object
 9   phone               0 non-null      object
 10  latitude            0 non-null      object
 11  longitude           0 non-null      object
 12  has_droppff         0 non-null      object
 13  has_phone_number    0 non-null      object
 14  county_website_url  0 non-null      object
 15  validate_url        0 non-null      object
 16  email               0 non-null      object


In [60]:
all_data_df['county'] = dropOff_df['county']
all_data_df['location_type'] = dropOff_df['location_type']
all_data_df['address_1'] = dropOff_df['address_1']
all_data_df['address_2'] = dropOff_df['address_2']
all_data_df['city'] = dropOff_df['city']
all_data_df['zip'] = dropOff_df['zip']
all_data_df['hours'] = dropOff_df['hours']
all_data_df['notes'] = dropOff_df['notes']
all_data_df['state'] = 'IL'
all_data_df['validate_url'] = 'https://www.elections.il.gov/VotingAndRegistrationSystems/VoteByMailBallotDropBoxLocations.aspx?MID=0DjRa0Y9V6U%3d&T=637369778439680154'

In [61]:
all_data_df.head()

,address_1,state,county,Field 23,location_type,address_2,city,state_2,zip,phone,...,has_phone_number,county_website_url,validate_url,email,fax,social,inactive,hours,rules,notes
0,507 Vermont ST,IL,Adams County,NaN,County Clerk's Office,NaN,Quincy,NaN,62301,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"September 24, 2020 and concludes November 3, 2...",NaN,"On November 3rd, drop box closed at 7:00 PM"
1,200 Court Street,IL,Brown County,NaN,Brown County Clerk’s Office,Room 4,Mt. Sterling,NaN,62353,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"September 24 - November 3, 2020 7:00 PM: 24 Hours",NaN,Right side of Courthouse Entrance
2,301 N Main Street,IL,Carroll County,NaN,Carroll County Courthouse,NaN,Mount Carroll,NaN,61053,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"September 24, 2020 thru November 3, 2020: 24 h...",NaN,Drop Box will be attached to building by the f...
3,100 E Springfield St,IL,Cass County,NaN,Cass County Clerk's Office,NaN,Virginia,NaN,62691,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"Begins on September 24, 2020 and concludes on ...",NaN,Attached to the building just to the right of ...
4,1776 E Washington St,IL,Champaign County,NaN,Champaign County Clerk's Office,NaN,Urbana,NaN,61802,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"Opens September 26, 2020: Open 24 Hours a Day,...",NaN,Ballots must be deposited by 7PM on Election D...


In [62]:
all_data_df.tail()

,address_1,state,county,Field 23,location_type,address_2,city,state_2,zip,phone,...,has_phone_number,county_website_url,validate_url,email,fax,social,inactive,hours,rules,notes
331,121 E Eighth ST,IL,Will County,NaN,White Oak Library,NaN,Lockport,NaN,60441,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"November 1, 2020: Sunday 1:00 PM to 5:00 PM",NaN,Right side of circulation desk inside main ent...
332,121 E Eighth ST,IL,Will County,NaN,White Oak Library,NaN,Lockport,NaN,60441,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"November 2, 2020: Monday 10:00 AM to 8:30 PM",NaN,Right side of circulation desk inside main ent...
333,115 N Main St,IL,Woodford County,NaN,County Clerk's Office,Rm 104,Eureka,NaN,61530,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"Begins on September 24, 2020 and concluding on...",NaN,
334,55 w Tompkins St,IL,City of Galesburg,NaN,City Election Office (City Hall),NaN,Galesburg,NaN,61401,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,"September 24th through November 3, 2020: 24/7",NaN,A secure ballot drop box is located on the Cit...
335,301 S 6th ST,IL,City of Rockford,NaN,Rockford Election Commssion Office,NaN,Rockford,NaN,61104,NaN,...,NaN,NaN,https://www.elections.il.gov/VotingAndRegistra...,NaN,NaN,NaN,NaN,October 1 thru November 3: 24 hrs. Ends Novemb...,NaN,City of Rockford Residents Only


# Write dropoff data to csv file

In [63]:
all_data_df.to_csv('Illinois.csv')

# Read csv of Illinois jurisdicitons
- The jurisidictions are just a table on the Illinois site so it was quicker to just copy them into a csv and clean them up here

In [195]:
juries_df = pd.read_csv('Illinois_jurisdictions.csv', header=1, index_col=None)

In [196]:
juries_df.head()

,Unnamed: 0,Jurisdiction,Name,Title,Email,Address,Phone,Fax
0,NaN,ADAMS,RYAN A. NIEKAMP,COUNTY CLERK,rniekamp@co.adams.il.us,"507 VERMONT\nQUINCY, IL 62306",(217) 277-2150,(217) 277-2155
1,NaN,ADAMS,RYAN A. NIEKAMP,ELECTION DIVISION,countyclerk@co.adams.il.us,"507 VERMONT\nQUINCY, IL 62306",(217) 277-2150,(217) 277-2155
2,NaN,ALEXANDER,ELLEN HENDERSON-BIGHAM,COUNTY CLERK,acc@alexandercountyil.org,"2000 WASHINGTON AVENUE\nCAIRO, IL 62914",(618) 734-7000,(618) 734-7002
3,NaN,BOND,MEG A. SYBERT,COUNTY CLERK,bococlerk@bondcountyil.com,"203 WEST COLLEGE\nGREENVILLE, IL 62246",(618) 664-0449,(618) 664-9414
4,NaN,BOONE,JULIE A. STAPLER,COUNTY CLERK,jstapler@boonecountyil.org,"1212 LOGAN AVENUE\nSUITE 103\nBELVIDERE, IL 61008",(815) 544-3103,(815) 547-8701


In [197]:
juries_df = juries_df.drop(['Unnamed: 0', 'Name', 'Title'], axis=1)

In [198]:
juries_df.head()

,Jurisdiction,Email,Address,Phone,Fax
0,ADAMS,rniekamp@co.adams.il.us,"507 VERMONT\nQUINCY, IL 62306",(217) 277-2150,(217) 277-2155
1,ADAMS,countyclerk@co.adams.il.us,"507 VERMONT\nQUINCY, IL 62306",(217) 277-2150,(217) 277-2155
2,ALEXANDER,acc@alexandercountyil.org,"2000 WASHINGTON AVENUE\nCAIRO, IL 62914",(618) 734-7000,(618) 734-7002
3,BOND,bococlerk@bondcountyil.com,"203 WEST COLLEGE\nGREENVILLE, IL 62246",(618) 664-0449,(618) 664-9414
4,BOONE,jstapler@boonecountyil.org,"1212 LOGAN AVENUE\nSUITE 103\nBELVIDERE, IL 61008",(815) 544-3103,(815) 547-8701


In [199]:
juries_df.drop_duplicates('Jurisdiction', inplace=True)

In [200]:
juries_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 139
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Jurisdiction  108 non-null    object
 1   Email         108 non-null    object
 2   Address       108 non-null    object
 3   Phone         108 non-null    object
 4   Fax           106 non-null    object
dtypes: object(5)
memory usage: 5.1+ KB


In [201]:
juries_df.rename(columns={'Jurisdiction':'county'}, inplace=True)

In [202]:
juries_df['county']=juries_df['county'].apply(lambda x: str.title(x))

In [203]:
juries_df['address_1'] = juries_df['Address'].apply(lambda x: x.splitlines()[0])

In [204]:
juries_df['zip']=juries_df['Address'].apply(lambda x: x.splitlines()[-1])

In [205]:
juries_df['city']=juries_df['zip'].apply(lambda x: x.split(',')[0])

In [206]:
juries_df['zip']=juries_df['zip'].apply(lambda x: x.split(' ')[-1])

In [207]:
juries_df['county']=juries_df['county'].apply(lambda x: x+' County' if x.split(' ')[0]!='City' else x) 

In [208]:
juries_df.drop('Address', axis=1, inplace=True)

In [210]:
juries_df['address_2']=juries_df['address_1'].apply(lambda x: x.split(', ')[1] if re.search(',', x) else np.nan)

In [211]:
juries_df['address_1']=juries_df['address_1'].apply(lambda x: x.split(', ')[0])

In [212]:
juries_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 139
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     108 non-null    object
 1   Email      108 non-null    object
 2   Phone      108 non-null    object
 3   Fax        106 non-null    object
 4   address_1  108 non-null    object
 5   zip        108 non-null    object
 6   city       108 non-null    object
 7   address_2  16 non-null     object
dtypes: object(8)
memory usage: 7.6+ KB


# Write jurisdiciton data to csv file

In [215]:
juries_df.to_csv('Illinois_juries.csv', index=False)